人臉偵測

In [1]:
import cv2
import mediapipe as mp
import random

cap = cv2.VideoCapture(0)
fps = cap.get(cv2.CAP_PROP_FPS)  # 視訊幀率
mp_face_detection = mp.solutions.face_detection  #設置mp_face_detection變數為MediaPipe的人臉檢測模型。
mp_drawing = mp.solutions.drawing_utils          # MediaPipe 繪圖工具。
mp_drawing_styles = mp.solutions.drawing_styles  # mediapipe 繪圖樣式
mp_hands = mp.solutions.hands                    # mediapipe 偵測手掌方法

with mp_face_detection.FaceDetection(
    model_selection=0, min_detection_confidence=0.5) as face_detection:

    if not cap.isOpened():
        print("Cannot open camera")
        exit()
    while True:
        ret, img = cap.read()
        if not ret:
            print("Cannot receive frame")
            break

        img.flags.writeable = False
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        results = face_detection.process(img)

        img.flags.writeable = True
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        if results.detections:            
            text = f"Number of faces: {len(results.detections)}"
            print(text)
            cv2.putText(img, text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
            for detection in results.detections:
                mp_drawing.draw_detection(img, detection)
        else :
            print("Number of faces: 0")
            text0 = f"Number of faces: 0"
            cv2.putText(img, text0, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

        cv2.imshow('oxxostudio', img)
        key = cv2.waitKey(round(800/fps)) & 0xFF
        if key == 27:
            break    # 按下 q 鍵停止
           
cap.release()
cv2.destroyAllWindows()

Number of faces: 1
Number of faces: 1
Number of faces: 1
Number of faces: 1
Number of faces: 1
Number of faces: 1
Number of faces: 1
Number of faces: 1
Number of faces: 1
Number of faces: 1
Number of faces: 1
Number of faces: 1
Number of faces: 1
Number of faces: 1
Number of faces: 1
Number of faces: 1
Number of faces: 1
Number of faces: 1
Number of faces: 1
Number of faces: 1
Number of faces: 1
Number of faces: 1
Number of faces: 1
Number of faces: 1
Number of faces: 1
Number of faces: 1
Number of faces: 1
Number of faces: 1
Number of faces: 1
Number of faces: 1
Number of faces: 1
Number of faces: 1
Number of faces: 1
Number of faces: 1
Number of faces: 1
Number of faces: 1
Number of faces: 1
Number of faces: 1
Number of faces: 1
Number of faces: 1
Number of faces: 1
Number of faces: 1
Number of faces: 1
Number of faces: 1
Number of faces: 1
Number of faces: 1
Number of faces: 1
Number of faces: 1
Number of faces: 1
Number of faces: 1
Number of faces: 1
Number of faces: 1
Number of fa

偵測手勢+畫畫

In [ ]:
import cv2
import mediapipe as mp
import math

mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_hands = mp.solutions.hands

# 根據兩點的座標，計算角度
def vector_2d_angle(v1, v2):
    v1_x = v1[0]
    v1_y = v1[1]
    v2_x = v2[0]
    v2_y = v2[1]
    try:
        angle_= math.degrees(math.acos((v1_x*v2_x+v1_y*v2_y)/(((v1_x**2+v1_y**2)**0.5)*((v2_x**2+v2_y**2)**0.5))))
    except:
        angle_ = 180
    return angle_

# 根據傳入的 21 個節點座標，得到該手指的角度
def hand_angle(hand_):
    angle_list = []
    # thumb 大拇指角度
    angle_ = vector_2d_angle(
        ((int(hand_[0][0])- int(hand_[2][0])),(int(hand_[0][1])-int(hand_[2][1]))),
        ((int(hand_[3][0])- int(hand_[4][0])),(int(hand_[3][1])- int(hand_[4][1])))
        )
    angle_list.append(angle_)
    # index 食指角度
    angle_ = vector_2d_angle(
        ((int(hand_[0][0])-int(hand_[6][0])),(int(hand_[0][1])- int(hand_[6][1]))),
        ((int(hand_[7][0])- int(hand_[8][0])),(int(hand_[7][1])- int(hand_[8][1])))
        )
    angle_list.append(angle_)
    # middle 中指角度
    angle_ = vector_2d_angle(
        ((int(hand_[0][0])- int(hand_[10][0])),(int(hand_[0][1])- int(hand_[10][1]))),
        ((int(hand_[11][0])- int(hand_[12][0])),(int(hand_[11][1])- int(hand_[12][1])))
        )
    angle_list.append(angle_)
    # ring 無名指角度
    angle_ = vector_2d_angle(
        ((int(hand_[0][0])- int(hand_[14][0])),(int(hand_[0][1])- int(hand_[14][1]))),
        ((int(hand_[15][0])- int(hand_[16][0])),(int(hand_[15][1])- int(hand_[16][1])))
        )
    angle_list.append(angle_)
    # pink 小拇指角度
    angle_ = vector_2d_angle(
        ((int(hand_[0][0])- int(hand_[18][0])),(int(hand_[0][1])- int(hand_[18][1]))),
        ((int(hand_[19][0])- int(hand_[20][0])),(int(hand_[19][1])- int(hand_[20][1])))
        )
    angle_list.append(angle_)
    return angle_list

# 根據手指角度的串列內容，返回對應的手勢名稱
def hand_pos(finger_angle):
    f1 = finger_angle[0]   # 大拇指角度
    f2 = finger_angle[1]   # 食指角度
    f3 = finger_angle[2]   # 中指角度
    f4 = finger_angle[3]   # 無名指角度
    f5 = finger_angle[4]   # 小拇指角度

    # 小於 50 表示手指伸直，大於等於 50 表示手指捲縮
    if f1>=50 and f2<50 and f3>=50 and f4>=50 and f5>=50:
        return '1'
    else:
        return ''

cap = cv2.VideoCapture(0)            # 讀取攝影機
fontFace = cv2.FONT_HERSHEY_SIMPLEX  # 印出文字的字型
lineType = cv2.LINE_AA               # 印出文字的邊框

# mediapipe 啟用偵測手掌
with mp_hands.Hands(
    model_complexity=0,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as hands:

    if not cap.isOpened():
        print("Cannot open camera")
        exit()
    w, h = 540, 310                                        # 影像尺寸
    draw = np.zeros((h,w,4), dtype='uint8')                # 繪製全黑背景，尺寸和影像相同
    dots = []                                              # 使用 dots 空串列記錄繪圖座標點
    cv2.rectangle(draw,(20,20),(60,60),(0,0,255,255),-1)   # 在畫面上方放入紅色正方形
    cv2.rectangle(draw,(80,20),(120,60),(0,255,0,255),-1)  # 在畫面上方放入綠色正方形
    cv2.rectangle(draw,(140,20),(180,60),(255,0,0,255),-1) # 在畫面上方放入藍色正方形
    color = (0,0,255,255)                                  # 設定預設顏色為紅色
    while True:
        ret, img = cap.read()
        img = cv2.resize(img, (w,h))                       # 縮小尺寸，加快處理效率
        img = cv2.flip(img, 1)
        if not ret:
            print("Cannot receive frame")
            break
        img2 = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)        # 偵測手勢的影像轉換成 RGB 色彩
        img = cv2.cvtColor(img, cv2.COLOR_BGR2BGRA)        # 畫圖的影像轉換成 BGRA 色彩
        results = hands.process(img2)                      # 偵測手勢
        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                finger_points = []                         # 記錄手指節點座標的串列
                for i in hand_landmarks.landmark:
                    # 將 21 個節點換算成座標，記錄到 finger_points
                    x = i.x*w
                    y = i.y*h
                    finger_points.append((x,y))
                if finger_points:
                    finger_angle = hand_angle(finger_points) # 計算手指角度，回傳長度為 5 的串列
                    text = hand_pos(finger_angle)            # 取得手勢所回傳的內容
                    if text == '1':
                        fx = int(finger_points[8][0])        # 如果手勢為 1，記錄食指末端的座標
                        fy = int(finger_points[8][1])
                        if fy>=20 and fy<=60 and fx>=20 and fx<=60:
                            color = (0,0,255,255)            # 如果食指末端碰到紅色，顏色改成紅色
                        elif fy>=20 and fy<=60 and fx>=80 and fx<=120:
                            color = (0,255,0,255)            # 如果食指末端碰到綠色，顏色改成綠色
                        elif fy>=20 and fy<=60 and fx>=140 and fx<=180:
                            color = (255,0,0,255)            # 如果食指末端碰到藍色，顏色改成藍色
                        else:
                            dots.append([fx,fy])             # 記錄食指座標
                            dl = len(dots)
                            if dl>1:
                                dx1 = dots[dl-2][0]
                                dy1 = dots[dl-2][1]
                                dx2 = dots[dl-1][0]
                                dy2 = dots[dl-1][1]
                                cv2.line(draw,(dx1,dy1),(dx2,dy2),color,5)  # 在黑色畫布上畫圖
                    else:
                        dots = [] # 如果換成別的手勢，清空 dots

        # 將影像和黑色畫布合成
        for j in range(w):
            img[:,j,0] = img[:,j,0]*(1-draw[:,j,3]/255) + draw[:,j,0]*(draw[:,j,3]/255)
            img[:,j,1] = img[:,j,1]*(1-draw[:,j,3]/255) + draw[:,j,1]*(draw[:,j,3]/255)
            img[:,j,2] = img[:,j,2]*(1-draw[:,j,3]/255) + draw[:,j,2]*(draw[:,j,3]/255)

        cv2.imshow('oxxostudio', img)
        keyboard = cv2.waitKey(round(800/fps)) & 0xFF
        if keyboard == 27:
            break
        # 按下 r 重置畫面
        if keyboard == ord('r'):
            draw = np.zeros((h,w,4), dtype='uint8')
            cv2.rectangle(draw,(20,20),(60,60),(0,0,255,255),-1)   # 在畫面上方放入紅色正方形
            cv2.rectangle(draw,(80,20),(120,60),(0,255,0,255),-1)  # 在畫面上方放入綠色正方形
            cv2.rectangle(draw,(140,20),(180,60),(255,0,0,255),-1) # 在畫面上方放入藍色正方形
cap.release()
cv2.destroyAllWindows()

手勢遊戲

In [2]:
import cv2
import mediapipe as mp
import random

mp_drawing = mp.solutions.drawing_utils          # mediapipe 繪圖方法
mp_drawing_styles = mp.solutions.drawing_styles  # mediapipe 繪圖樣式
mp_hands = mp.solutions.hands                    # mediapipe 偵測手掌方法

cap = cv2.VideoCapture(0)

# mediapipe 啟用偵測手掌
with mp_hands.Hands(
    model_complexity=0,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as hands:

    if not cap.isOpened():
        print("Cannot open camera")
        exit()

    run = True         # 設定是否更動觸碰區位置
    while True:
        ret, img = cap.read()
        if not ret:
            print("Cannot receive frame")
            break
        img = cv2.resize(img,(540,320))  # 調整畫面尺寸
        size = img.shape   # 取得攝影機影像尺寸
        w = size[1]        # 取得畫面寬度
        h = size[0]        # 取得畫面高度
        if run:
            run = False    # 如果沒有碰到，就一直是 False ( 不會更換位置 )
            rx = random.randint(50,w-50)    # 隨機 x 座標
            ry = random.randint(50,h-100)   # 隨機 y 座標
            print(rx, ry)
                     
        img2 = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)   # 將 BGR 轉換成 RGB
        results = hands.process(img2)                 # 偵測手掌
        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                x = hand_landmarks.landmark[7].x * w   # 取得食指末端 x 座標
                y = hand_landmarks.landmark[7].y * h   # 取得食指末端 y 座標
                print(x,y)
                if x>rx and x<(rx+80) and y>ry and y<(ry+80):
                    run = True
                # 將節點和骨架繪製到影像中
                mp_drawing.draw_landmarks(
                    img,
                    hand_landmarks,
                    mp_hands.HAND_CONNECTIONS,
                    mp_drawing_styles.get_default_hand_landmarks_style(),
                    mp_drawing_styles.get_default_hand_connections_style())

        cv2.rectangle(img,(rx,ry),(rx+80,ry+80),(0,0,255),5)   # 畫出觸碰區
        cv2.imshow('Report', img)
        key = cv2.waitKey(round(800/fps)) & 0xFF
        if key == 27:
            break    # 按下 q 鍵停止
cap.release()
cv2.destroyAllWindows()

53 171
434.15968894958496 45.0864315032959


NameError: name 'fps' is not defined

In [1]:
import cv2
from deepface import DeepFace
import numpy as np
from PIL import ImageFont, ImageDraw, Image

# 定義該情緒的中文字
text_obj={
    'angry': '生氣',
    'disgust': '噁心',
    'fear': '害怕',
    'happy': '開心',
    'sad': '難過',
    'surprise': '驚訝',
    'neutral': '正常'
}

# 定義加入文字函式
def putText(x,y,text,size=50,color=(255,255,255)):
    global img
    fontpath = 'NotoSansTC-Regular.otf'            # 字型
    font = ImageFont.truetype(fontpath, size)      # 定義字型與文字大小
    imgPil = Image.fromarray(img)                  # 轉換成 PIL 影像物件
    draw = ImageDraw.Draw(imgPil)                  # 定義繪圖物件
    draw.text((x, y), text_obj[text], fill=color, font=font) # 加入文字
    img = np.array(imgPil)                         # 轉換成 np.array

cap = cv2.VideoCapture(0)
# 讀取視訊參數
height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)  # 視訊畫面高
width = cap.get(cv2.CAP_PROP_FRAME_WIDTH)  # 視訊畫面寬
fps = cap.get(cv2.CAP_PROP_FPS)  # 視訊幀率
total_count = cap.get(cv2.CAP_PROP_FRAME_COUNT)  # 視訊總幀數
print("影像高,寬: %d x %d" % (height, width))
print("視訊幀率(FPS, Frame Per Second): %.2f" % fps)
print("視訊總幀數: %d" % total_count)

if not cap.isOpened():
    print("Cannot open camera")
    exit()
while True:
    ret, frame = cap.read()
    if not ret:
        print("Cannot receive frame")
        break
    img = cv2.resize(frame,(384,240))
    try:
        analyze = DeepFace.analyze(img, actions=['emotion'])
        emotion = analyze['dominant_emotion']  # 取得情緒文字
        putText(0,40,emotion)                  # 放入文字
    except:
        pass
    cv2.imshow('emo', img)
    key = cv2.waitKey(round(800/fps)) & 0xFF
    if key == 27 :
        break
cap.release()
cv2.destroyAllWindows()

影像高,寬: 480 x 640
視訊幀率(FPS, Frame Per Second): 30.00
視訊總幀數: -1


Action: emotion: 100%|██████████| 1/1 [00:00<00:00, 13.55it/s]


臉部偵測+手勢遊戲

In [2]:
import cv2
import mediapipe as mp
import random
import numpy as np

cap = cv2.VideoCapture(0)
fps = cap.get(cv2.CAP_PROP_FPS)  # 視訊幀率
mp_face_detection = mp.solutions.face_detection
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

with mp_face_detection.FaceDetection(
    model_selection=0, min_detection_confidence=0.5) as face_detection, \
    mp_hands.Hands(
    model_complexity=0, min_detection_confidence=0.5, min_tracking_confidence=0.5) as hands:

    if not cap.isOpened():
        print("Cannot open camera")
        exit()

    run = True
    while True:
        ret, img = cap.read()
        if not ret:
            print("Cannot receive frame")
            break
        img = cv2.resize(img, (720, 320))
        size = img.shape
        w = size[1]
        h = size[0]
        if run:
            run = False
            rx = random.randint(50, w - 50)
            ry = random.randint(50, h - 100)
            #print(rx, ry)
            print(f"圖形座標: ", rx,ry)
        img.flags.writeable = False
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        results_face = face_detection.process(img)
        results_hands = hands.process(img)

        img.flags.writeable = True
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        
        #img2 = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        #results_face = face_detection.process(img2)
        #results_hands = hands.process(img2)
        if results_face.detections:
            textf = f"Number of faces: {len(results_face.detections)}"
            print(textf)
            cv2.putText(img, textf, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)            
            for detection in results_face.detections:
                mp_drawing.draw_detection(img, detection)
        else :
            print("Number of faces: 0")
            textf0 = f"Number of faces: 0"
            cv2.putText(img, textf0, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
        
        if results_hands.multi_hand_landmarks:
            hand_count = f"Number of hands:{len(results_hands.multi_hand_landmarks)} " # 手的數量
            cv2.putText(img, hand_count, (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)   
            for hand_landmarks in results_hands.multi_hand_landmarks:
                x = hand_landmarks.landmark[7].x * w
                y = hand_landmarks.landmark[7].y * h
                print(x, y)
                if x > rx and x < (rx + 80) and y > ry and y < (ry + 80):
                    run = True
                mp_drawing.draw_landmarks(
                    img,
                    hand_landmarks,
                    mp_hands.HAND_CONNECTIONS,
                    mp_drawing_styles.get_default_hand_landmarks_style(),
                    mp_drawing_styles.get_default_hand_connections_style())
        else :
            print("Number of hands: 0")
            hand_count0 = f"Number of hands: 0"
            cv2.putText(img, hand_count0, (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
        
        #cv2.rectangle(img, (rx, ry), (rx + 80, ry + 80), (0, 0, 255), 5)
        triangle_pts = np.array([[rx + 40, ry], [rx, ry + 80], [rx + 80, ry + 80]], np.int32)
        cv2.polylines(img, [triangle_pts], True, (0, 0, 255), thickness=5)
        
        cv2.imshow('Game', img)
        key = cv2.waitKey(round(800/fps)) & 0xFF
        if key == 27:
            break
cap.release()
cv2.destroyAllWindows()

圖形座標:  66 207
Number of faces: 1
574.8235702514648 156.3170337677002
Number of faces: 1
567.7754974365234 159.6666431427002
Number of faces: 1
568.8213872909546 164.4009017944336
Number of faces: 1
569.2062091827393 161.81262969970703
Number of faces: 1
569.8036336898804 154.34818267822266
Number of faces: 1
572.048749923706 158.99015426635742
Number of faces: 1
571.0910511016846 159.9149227142334
Number of faces: 1
572.0037317276001 161.9906234741211
Number of faces: 1
570.3860807418823 157.2331428527832
Number of faces: 1
571.0212278366089 157.64148712158203
Number of faces: 1
573.768196105957 162.35218048095703
Number of faces: 1
570.8019304275513 157.020902633667
Number of faces: 1
573.109016418457 151.93904876708984
Number of faces: 1
576.7902946472168 153.18069458007812
Number of faces: 1
577.3108148574829 150.5126190185547
Number of faces: 1
586.7706441879272 165.57586669921875
Number of faces: 1
582.2154808044434 163.24813842773438
Number of faces: 1
575.1221323013306 162.68213

In [4]:
import cv2
import mediapipe as mp

cap = cv2.VideoCapture(0)
mp_face_detection = mp.solutions.face_detection
mp_drawing = mp.solutions.drawing_utils

# 設定錄影參數
fps = 30
output_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
output_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
output_filename = "D:/OBS/test.mp4 影片"
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
output_video = cv2.VideoWriter(output_filename, fourcc, fps, (output_width, output_height))

with mp_face_detection.FaceDetection(
    model_selection=0, min_detection_confidence=0.5) as face_detection:

    if not cap.isOpened():
        print("Cannot open camera")
        exit()

    while True:
        ret, img = cap.read()
        if not ret:
            print("Cannot receive frame")
            break

        img.flags.writeable = False
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        results = face_detection.process(img)

        img.flags.writeable = True
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

        num_faces = 0
        if results.detections:
            num_faces = len(results.detections)
            for detection in results.detections:
                mp_drawing.draw_detection(img, detection)

        # 顯示人臉數量
        text = f"Number of faces: {num_faces}"
        cv2.putText(img, text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

        # 寫入影格至輸出影片
        output_video.write(img)

        cv2.imshow('Face Detection', img)
        key = cv2.waitKey(1) & 0xFF
        if key == 27:
            break

cap.release()
output_video.release()
cv2.destroyAllWindows()


In [3]:
!python --version

Python 3.9.16
